In [25]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import KNNImputer

In [26]:
train_df = pd.read_csv('./inputs/train.csv')
target = train_df["rainfall"]
train_df.drop("day", inplace=True, axis=1)
train_df.drop("rainfall", inplace=True, axis=1)
train_df

,id,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
0,0,1017.4,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,17.2
1,1,1019.5,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,21.9
2,2,1024.1,19.4,16.1,14.6,9.3,75.0,47.0,8.3,70.0,18.1
3,3,1013.4,18.1,17.8,16.9,16.8,95.0,95.0,0.0,60.0,35.6
4,4,1021.8,21.3,18.4,15.2,9.6,52.0,45.0,3.6,40.0,24.8
...,...,...,...,...,...,...,...,...,...,...,...
2185,2185,1014.6,23.2,20.6,19.1,19.9,97.0,88.0,0.1,40.0,22.1
2186,2186,1012.4,17.2,17.3,16.3,15.3,91.0,88.0,0.0,50.0,35.3
2187,2187,1013.3,19.0,16.3,14.3,12.6,79.0,79.0,5.0,40.0,32.9
2188,2188,1022.3,16.4,15.2,13.8,14.7,92.0,93.0,0.1,40.0,18.0


In [27]:
test_df = pd.read_csv('./inputs/test.csv')
test_df.drop("day", inplace=True, axis=1)
test_df

,id,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
0,2190,1019.5,17.5,15.8,12.7,14.9,96.0,99.0,0.0,50.0,24.3
1,2191,1016.5,17.5,16.5,15.8,15.1,97.0,99.0,0.0,50.0,35.3
2,2192,1023.9,11.2,10.4,9.4,8.9,86.0,96.0,0.0,40.0,16.9
3,2193,1022.9,20.6,17.3,15.2,9.5,75.0,45.0,7.1,20.0,50.6
4,2194,1022.2,16.1,13.8,6.4,4.3,68.0,49.0,9.2,20.0,19.4
...,...,...,...,...,...,...,...,...,...,...,...
725,2915,1020.8,18.2,17.6,16.1,13.7,96.0,95.0,0.0,20.0,34.3
726,2916,1011.7,23.2,18.1,16.0,16.0,78.0,80.0,1.6,40.0,25.2
727,2917,1022.7,21.0,18.5,17.0,15.5,92.0,96.0,0.0,50.0,21.9
728,2918,1014.4,21.0,20.0,19.7,19.8,94.0,93.0,0.0,50.0,39.5


In [28]:
knn_imputer = KNNImputer(n_neighbors=2)
df_knn_imputed = pd.DataFrame(knn_imputer.fit_transform(test_df), columns=test_df.columns)
test_df = df_knn_imputed

In [29]:
scaler = StandardScaler()
scaler.fit(train_df)
train_scaled = scaler.transform(train_df)
test_scaled = scaler.transform(test_df)

In [30]:
X_train = train_scaled
y_train = target
X_test = test_scaled

In [38]:
clf = CatBoostClassifier(n_estimators=1000, learning_rate=0.01, max_depth=5, random_state=42)
clf.load_model("./outputs/catboost.model")
clf.fit(X_train,y_train,init_model=clf)
clf.save_model("./outputs/catboost.model")
predictions = clf.predict(X_test)
predictions

0:	learn: 0.2472195	total: 1.94ms	remaining: 1.94s
1:	learn: 0.2471723	total: 3.97ms	remaining: 1.98s
2:	learn: 0.2471119	total: 5.53ms	remaining: 1.84s
3:	learn: 0.2470755	total: 7.09ms	remaining: 1.76s
4:	learn: 0.2470370	total: 8.65ms	remaining: 1.72s
5:	learn: 0.2469720	total: 10.3ms	remaining: 1.71s
6:	learn: 0.2468442	total: 12.5ms	remaining: 1.78s
7:	learn: 0.2467471	total: 14.2ms	remaining: 1.76s
8:	learn: 0.2466147	total: 15.8ms	remaining: 1.74s
9:	learn: 0.2465549	total: 17.5ms	remaining: 1.73s
10:	learn: 0.2465252	total: 19.2ms	remaining: 1.72s
11:	learn: 0.2464164	total: 20.8ms	remaining: 1.71s
12:	learn: 0.2462840	total: 22.5ms	remaining: 1.71s
13:	learn: 0.2462036	total: 24.3ms	remaining: 1.71s
14:	learn: 0.2461458	total: 26ms	remaining: 1.7s
15:	learn: 0.2460205	total: 27.8ms	remaining: 1.71s
16:	learn: 0.2459254	total: 29.7ms	remaining: 1.72s
17:	learn: 0.2458274	total: 31.3ms	remaining: 1.71s
18:	learn: 0.2457057	total: 33ms	remaining: 1.7s
19:	learn: 0.2456430	total: 

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [40]:
submission_test = pd.read_csv("./inputs/test.csv")
output = pd.DataFrame({'id': submission_test.id, 'rainfall': predictions})
output.to_csv('./outputs/submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [41]:
# !kaggle competitions submit -c playground-series-s5e3 -f ./outputs/submission.csv -m f"{clf.__class__.__name__} with {clf.get_param("n_estimators")} estimators, a learning rate of {clf.get_param("learning_rate")}, max depth of {clf.get_param("max_depth")} and a random state of {clf.get_param("random_state")}"

Successfully submitted to Binary Prediction with a Rainfall Dataset



  0%|          | 0.00/5.72k [00:00<?, ?B/s]
100%|##########| 5.72k/5.72k [00:00<00:00, 28.4kB/s]
100%|##########| 5.72k/5.72k [00:00<00:00, 7.94kB/s]


In [34]:
model = LogisticRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
predictions

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [35]:
# !kaggle competitions submit -c playground-series-s5e3 -f ./outputs/submission.csv -m f"{clf.__class__.__name__} with {clf.get_param("n_estimators")} estimators, a learning rate of {clf.get_param("learning_rate")}, max depth of {clf.get_param("max_depth")} and a random state of {clf.get_param("random_state")}"

Successfully submitted to Binary Prediction with a Rainfall Dataset



  0%|          | 0.00/5.72k [00:00<?, ?B/s]
100%|##########| 5.72k/5.72k [00:00<00:00, 11.7kB/s]
100%|##########| 5.72k/5.72k [00:01<00:00, 5.64kB/s]
